# Data Preprocess

In [2]:
import os
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
import json
import torch
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import category_encoders as ce
#from torchvision import models
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,classification_report,roc_auc_score
#from torchvision import transforms
import torch.nn.functional as F
import torch.nn as nn
import utils

import gc

In [3]:
# train_X = pd.read_csv('music_data/train_X.csv')
train_X = pd.read_csv('music_data/train_X_date_converted.csv')
train_y = pd.read_csv('music_data/train_Y.csv')

# val_X = pd.read_csv('music_data/valid_X.csv')
val_X = pd.read_csv('music_data/valid_X_date_converted.csv')
val_y = pd.read_csv('music_data/valid_Y.csv')

# test_X = pd.read_csv('music_data/test_X.csv')
test_X = pd.read_csv('music_data/test_X_date_converted.csv')
test_y = pd.read_csv('music_data/test_Y.csv')

# # train_tiny_X = pd.read_csv('music_data/train_tiny_X.csv')
# train_tiny_X = pd.read_csv('music_data/train_tiny_X_date_converted.csv')
# train_tiny_y = pd.read_csv('music_data/train_tiny_Y.csv')



In [4]:
train_tiny_X=train_X[:10000].copy()
train_tiny_y = train_y[:10000].copy()

In [5]:
# val_tiny_X = val_X[:1000].copy()
val_tiny_X = val_X[:1024].copy()
val_tiny_y = val_y[:1024].copy()

In [6]:
train_y.target.iloc[3]

1.0

In [7]:
catCols = ['msno', 'song_id', 'source_screen_name', 'source_system_tab', 'source_type', 'genre_ids', 'artist_name', 
           'composer', 'lyricist', 'language', 'city', 'gender', 'registered_via']
numCols = ['Unnamed: 0','Unnamed: 0.1','bd', 'song_length','time','registration_init_time_int','expiration_date_int']

In [35]:
# train_X_catCols = train_tiny_X[catCols].values
# train_X_catCols = OneHotEncoder().fit_transform(train_X_catCols)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [8]:
train_X_numCols = train_tiny_X[numCols].values
print(train_X_numCols.shape)
train_X_numCols = StandardScaler().fit_transform(train_X_numCols)
print(train_X_numCols.shape)
train_tiny_X[numCols] = train_X_numCols

(10000, 7)
(10000, 7)


In [9]:
train_tiny_X.head()

,Unnamed: 0,Unnamed: 0.1,msno,song_id,source_screen_name,source_system_tab,source_type,song_length,genre_ids,artist_name,composer,lyricist,language,city,bd,gender,registered_via,time,registration_init_time_int,expiration_date_int
0,-1.731878,-0.356394,8145,253733,16,6,8,0.428309,371,4252,8389,2681,9,0,-0.650309,2,1,-0.356394,1.201362,-2.210220
1,-1.731531,0.553126,5224,145235,16,6,8,-0.796309,371,34892,74276,26024,6,3,0.873194,1,2,0.553126,0.119565,0.005168
2,-1.731185,1.359821,5474,22231,11,0,7,-0.563420,371,20609,27775,9110,9,0,-0.650309,2,1,1.359821,1.162370,-0.304510
3,-1.730838,-1.189612,23177,70181,8,3,3,0.331818,371,44425,83027,34734,2,0,-0.650309,2,0,-1.189612,0.283693,0.187798
4,-1.730492,-0.767353,3269,128141,12,2,2,1.219789,371,42400,81151,32836,2,0,-0.650309,2,2,-0.767353,-0.430856,-1.582924


In [10]:
# train_X_catCols = train_tiny_X[catCols]
# train_X_catCols = OneHotEncoder().fit_transform(train_X_catCols)
# print(train_X_catCols.shape,train_X_numCols.shape)

In [11]:
# train_X_catCols = np.array(train_X_catCols)
# train_X_numCols = np.array(train_X_numCols)
# print(train_X_catCols)
#train_all = np.concatenate((train_X_numCols,train_X_catCols),1)

In [10]:
ohe = ce.OneHotEncoder(handle_unknown='ignore', use_cat_names=True)
train_X_catCols = ohe.fit_transform(train_tiny_X[catCols])
train_X_catCols.head()

,msno,song_id,source_screen_name,source_system_tab,source_type,genre_ids,artist_name,composer,lyricist,language,city,gender,registered_via
0,8145,253733,16,6,8,371,4252,8389,2681,9,0,2,1
1,5224,145235,16,6,8,371,34892,74276,26024,6,3,1,2
2,5474,22231,11,0,7,371,20609,27775,9110,9,0,2,1
3,23177,70181,8,3,3,371,44425,83027,34734,2,0,2,0
4,3269,128141,12,2,2,371,42400,81151,32836,2,0,2,2


In [11]:
print(train_X_catCols.shape)

(10000, 13)


In [12]:
train_tiny_X.head()

,Unnamed: 0,Unnamed: 0.1,msno,song_id,source_screen_name,source_system_tab,source_type,song_length,genre_ids,artist_name,composer,lyricist,language,city,bd,gender,registered_via,time,registration_init_time_int,expiration_date_int
0,-1.731878,-0.356394,8145,253733,16,6,8,0.428309,371,4252,8389,2681,9,0,-0.650309,2,1,-0.356394,1.201362,-2.210220
1,-1.731531,0.553126,5224,145235,16,6,8,-0.796309,371,34892,74276,26024,6,3,0.873194,1,2,0.553126,0.119565,0.005168
2,-1.731185,1.359821,5474,22231,11,0,7,-0.563420,371,20609,27775,9110,9,0,-0.650309,2,1,1.359821,1.162370,-0.304510
3,-1.730838,-1.189612,23177,70181,8,3,3,0.331818,371,44425,83027,34734,2,0,-0.650309,2,0,-1.189612,0.283693,0.187798
4,-1.730492,-0.767353,3269,128141,12,2,2,1.219789,371,42400,81151,32836,2,0,-0.650309,2,2,-0.767353,-0.430856,-1.582924


In [13]:
class myDataset(Dataset):
    def __init__(self, dfX, dfy):
        #'Initialization'
        self.dfX = dfX
        self.dfy = dfy
#         self.numCol = numCol
#         self.catCol = catCol
    def __len__(self):
        #'Denotes the total number of samples'
        return len(self.dfX)
#     def transform(self,dfX,numCol):
#         X_num  = dfX[numCol].values
#         X_num = StandardScaler().fit_transform(X_num)
#         dfX[numCol] = X_num
#         return dfX
    def __getitem__(self, index):
        #'Generates one sample of data'
        # Load data and get label
#         dfX_t = self.transform(self.dfX,self.numCol)
        X = self.dfX.iloc[index]
        X = torch.tensor(X)
        y = self.dfy.target.iloc[index]
        y = torch.tensor(y)
        return X, y

In [14]:
torch.tensor(train_X.iloc[1])


tensor([1.0000e+00, 4.8755e+06, 5.2240e+03, 1.4524e+05, 1.6000e+01, 6.0000e+00,
        8.0000e+00, 2.0062e+05, 3.7100e+02, 3.4892e+04, 7.4276e+04, 2.6024e+04,
        6.0000e+00, 3.0000e+00, 4.1000e+01, 1.0000e+00, 2.0000e+00, 4.9078e-01,
        5.0000e+03, 6.4750e+03])

## Neural Network

In [15]:
BATCH_SIZE = 16
MAX_EPOCH = 20
SAVE_THRESHOLD = 0.6
SAVE_RESULTS = True
SAVE_LOGITS = False
SEED = 229
LOSS_HIST_ACCUM_FREQ = 100

In [16]:
# random.seed(SEED)
# np.random.seed(SEED)
# torch.manual_seed(SEED)
# torch.cuda.manual_seed_all(SEED)

In [17]:
#model
model = nn.Sequential(
    nn.Linear(20,40),
    nn.BatchNorm1d(40),
    nn.Dropout(p=0.5),
    nn.ReLU(),
    nn.Linear(40,20),
    nn.BatchNorm1d(20),
    nn.ReLU(),
    nn.Linear(20,1)
)

In [18]:
device, gpu_ids = utils.get_available_devices()
print(device,gpu_ids)
#model = nn.DataParallel(model, gpu_ids)
model = model.to(device)

cuda:0 [0]


In [19]:
# Optimizer
#criterion = torch.nn.BCEWithLogitsLoss()   # default
#criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=L2_PENALTY)
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=4, min_lr=3e-5, verbose=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=4, 
                                                       min_lr=1e-4, verbose=True)


In [20]:
def test(model, test_loader):
    model.eval();
    test_loss = 0.0
    true_ans_list = []
    preds_cat = []
    logits_list = []

    with torch.no_grad():
        for step, (features, targets) in enumerate(test_loader):
            features = features.to(device=device) # move to device, e.g. GPU
            targets = targets.to(device=device)
            logits = model(features)
            logits = torch.squeeze(logits,1)  
            loss = F.binary_cross_entropy(F.sigmoid(logits), targets)
            test_loss += loss.item()
            true_ans_list.append(targets)
            preds_cat.append(logits)
            logits=logits.cpu().numpy()
            logits_list.append(logits)

        all_true_ans = torch.cat(true_ans_list)
        all_preds = torch.cat(preds_cat)
        all_preds = torch.ge(all_preds, 0.5)
        all_true_ans=all_true_ans.cpu()
        all_preds=all_preds.cpu()
        f1_eval = f1_score(all_true_ans, all_preds, average='weighted')
        # drop the last logits_list
        logits_list = logits_list[:-1]
        # print("list:", logits_list)
        all_logits = np.stack(logits_list)

    return all_logits, all_true_ans, all_preds


def validate(model, valid_loader):
    model.eval();
    test_loss = 0.0
    true_ans_list = []
    preds_cat = []

    with torch.no_grad():
        valid_iterator = valid_loader

        for step, (features, targets) in enumerate(valid_iterator):
            features = features.to(device=device) # move to device, e.g. GPU
            targets = targets.to(device=device)
            logits = model(features)
            logits = torch.squeeze(logits,1)  
            loss = F.binary_cross_entropy(F.sigmoid(logits), targets)
            test_loss += loss.item()
            true_ans_list.append(targets)
            preds_cat.append(logits)

        all_true_ans = torch.cat(true_ans_list)
        all_preds = torch.cat(preds_cat)
        all_preds = torch.ge(all_preds, 0.5)
        all_true_ans=all_true_ans.cpu()
        all_preds=all_preds.cpu()
        # f1_eval = f1_score(all_true_ans, all_preds).item()
        f1_eval = f1_score(all_true_ans, all_preds, average='weighted')

    return test_loss / (step + 1), f1_eval



In [21]:
def cuda(x):
    return x.cuda(non_blocking=True)

### NN - Draft & Debugging on tiny dataset

In [22]:
train_dataset_tiny = myDataset(train_tiny_X,train_tiny_y)
val_dataset_tiny = myDataset(val_tiny_X, val_tiny_y)

In [23]:
train_tiny_loader = DataLoader(train_dataset_tiny, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_tiny_loader = DataLoader(val_dataset_tiny, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

In [25]:
train_loss_history = []
train_epoch_losses = []
valid_losses = []
valid_f1s = []
best_model_f1 = 0.0
best_model = None
best_model_ep = 0
num_ = len(os.listdir('output/')) + 1

for epoch in range(1, MAX_EPOCH + 1):
    #log.info('Starting epoch {}...'.format(epoch))
    print('starting Epoch:',epoch)
    model.train()
    total_loss = 0.0
    #train_tqdm = tqdm(train_tiny_loader)
    train_loss_curr_epoch = []

    for step, (features, targets) in enumerate(train_tiny_loader):
        features = features.to(device=device) # move to device, e.g. GPU
        #features = features.requires_grad_()
        targets = targets.to(device=device)
        optimizer.zero_grad()
        logits = model(features)
        logits = torch.squeeze(logits,1)        
        loss = F.binary_cross_entropy(torch.sigmoid(logits), targets)
        loss.backward()

#         nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optimizer.step()
        total_loss += loss.item()

        if (step+1) % LOSS_HIST_ACCUM_FREQ == 0:
            tr_loss_ = total_loss / step
            #tr_loss_= loss.item()
            train_loss_history.append(tr_loss_)
            train_loss_curr_epoch.append(tr_loss_)

    avg_epoch_loss = np.mean(train_loss_curr_epoch)
    print('mean train loss:', avg_epoch_loss)
    train_epoch_losses.append(avg_epoch_loss)

    print('Evaluating...')
    valid_loss, valid_f1 = validate(model, val_tiny_loader)
    print('Validation loss:',(round(valid_loss, 6)))
    print('Validation F1: ',(round(valid_f1, 5)))
    valid_losses.append(valid_loss)

    valid_f1s.append(valid_f1)
    scheduler.step(valid_loss)

    if valid_f1 >= best_model_f1:
        best_model = model
        best_model_f1 = valid_f1
        best_model_ep = epoch

    if (epoch > 3) and (epoch % 4 == 0) and best_model_f1 > SAVE_THRESHOLD:
        #log.info('Saving best model ({})...'.format(num_))
        model_pt = 'output/best_model{}.pt'.format(num_)
        torch.save(best_model, model_pt)

    if (epoch > 3) and SAVE_RESULTS:
        results_dict = dict()
        results_dict['train_loss_history'] = train_loss_history
        results_dict['valid_loss'] = valid_losses
        results_dict['valid_f1'] = valid_f1s
        results_path = 'output/results{}.json'.format(num_)
        with open(results_path, 'w') as fp:
            json.dump(results_dict, fp, indent=4)

    if (epoch > 7) and (utils.check_early_stopping(valid_losses, patience=6, threshold=3e-3)):
        #log.info('Early stopping...')
        break

print('Best F1 is',round(best_model_f1, 5), 'on epoch',best_model_ep)


# Save model (remember to change model name)
if best_model_f1 > SAVE_THRESHOLD:
    #log.info('Saving best model...')
    torch.save(best_model, 'output/best_model{}.pt'.format(num_))

if SAVE_LOGITS:
    #log.info('Saving test set info...')
    test_logits, test_true_ans, test_preds = test(best_model, test_loader)
    # test_logits = test_logits.cpu().numpy()
    test_true_ans = test_true_ans.cpu().numpy()
    test_preds = test_preds.cpu().numpy()
    test_path = 'output/test_results{}.npz'.format(num_)
    np.savez(test_path, test_logits=test_logits, test_true_ans=test_true_ans, test_preds=test_preds)

# Load weights
if LOAD_MODEL:
    model_path = 'output/best_model3.pt'
    #log.info('Loading weights from {}...'.format(model_path))

    # model.load_state_dict(torch.load('output/saved/best_model1.pt'))
    model = torch.load(model_path)
    model.to(device)
    # model.eval()
    # valid_loss, valid_f1 = validate(model, test_loader, criterion, need_tqdm=False)

if SAVE_PLOT:
    xs = list(range(1, len(train_epoch_losses) + 1))
    plt.title('Loss')
    plt.plot(xs, train_epoch_losses, '-o', label = 'train')
    plt.plot(xs, valid_losses, '-o', label = 'valid')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='upper right')
    plt.savefig('output/train_val_loss_dense{}.png'.format(num_))
    plt.show()
    plt.close()

    plt.title('F1')
    plt.plot(xs, valid_f1s, '-o')
    plt.ylabel('Valid F1')
    plt.xticks(xs);
    plt.xlabel('epoch');
    plt.savefig('output/valid_f1{}.png'.format(num_))
    plt.show()


starting Epoch: 1
mean train loss: 0.7017092737017032
Evaluating...


/opt/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Validation loss: 0.680793
Validation F1:  0.30974
starting Epoch: 2


/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


mean train loss: 0.6998481951739098
Evaluating...
Validation loss: 0.703721
Validation F1:  0.30974
starting Epoch: 3
mean train loss: 0.6987497721024045
Evaluating...
Validation loss: 0.710407
Validation F1:  0.30974
starting Epoch: 4
mean train loss: 0.6994790964679293
Evaluating...
Validation loss: 0.689664
Validation F1:  0.30974
starting Epoch: 5
mean train loss: 0.6980540488250236
Evaluating...
Validation loss: 0.683016
Validation F1:  0.30974
starting Epoch: 6
mean train loss: 0.6981917588898279
Evaluating...
Validation loss: 0.714533
Validation F1:  0.30974
Epoch     5: reducing learning rate of group 0 to 5.0000e-02.
starting Epoch: 7
mean train loss: 0.6976540876445766
Evaluating...
Validation loss: 0.680625
Validation F1:  0.30974
starting Epoch: 8
mean train loss: 0.6972023834266597
Evaluating...
Validation loss: 0.703615
Validation F1:  0.30974
starting Epoch: 9
mean train loss: 0.697888454493376
Evaluating...
Validation loss: 0.704461
Validation F1:  0.30974
starting Epoc

NameError: name 'LOAD_MODEL' is not defined

### NN - Full dataset w/o tunning

### NN - Tunning